In [131]:
import pandas as pd
import numpy as np
from typing import List, Tuple

In [157]:
# Parameters
spot_filename:str = "/tmp/index.csv"
options_filename:str = "../../data/nifty_options_eod.h5"
start_day:int = 4
de:int = 0
step:int = 50 
options:Tuple[Tuple[str, float]] = (
    ('p', 0),
    ('p', 0.01),
    ('c', 0)
)

In [158]:
index = pd.read_csv(spot_filename, parse_dates=['date'])
opt = pd.read_hdf(options_filename).sort_values(by=['date'])
rename = {
    'open_price': 'open',
    'high_price': 'high',
    'low_price': 'low',
    'close_pric': 'close'
}
opt = opt.rename(columns=rename)
opt['de'] = (opt['expiry_date'] - opt['date']).dt.days

In [159]:
ce = opt.query("opt_type=='CE'")
pe = opt.query("opt_type=='PE'")

In [160]:
def get_result(options_data, spot_data, opt='p', wkday=4, de=0, itm=0, step=100, holding_days=7):
    weekly = spot_data.query(f"wkday=={wkday}").copy()
    if opt.lower() == 'p':
        m = 1-itm
    else:
        m = 1+itm
    weekly['strike'] = [int((x*m)/step)*step for x in weekly.close.values]
    if opt.lower() == 'p':
        weekly['strike'] = weekly['strike'] + step
    opts = options_data.merge(weekly[['date', 'close', 'strike']].rename(
        columns={'close':'spot'}), on=['date', 'strike'])
    opts = opts.query(f"de>{de}")
    opts = opts.sort_values(by='de').groupby('date').first().reset_index()
    cols = ['date', 'contract_d', 'strike', 'expiry_date', 'close', 'spot']
    entries = opts[cols].copy()
    entries['entry_date'] = entries.date.copy()
    exit_frame = entries.copy()
    exit_frame['date'] = exit_frame.date + pd.DateOffset(days=holding_days)
    del exit_frame['close'] # Since we need the other close in expiry
    del exit_frame['contract_d'] 
    del exit_frame['spot']
    opt2 = exit_frame.merge(options_data, on=['date', 'strike', 'expiry_date'])
    opt2 = opt2.merge(weekly[['date', 'close']].rename(
        columns={'close': 'spot'}), on=['date'])
    exits = opt2[cols + ['entry_date']]
    exits = exits.rename(columns={'date':'exit_date', 'entry_date': 'date'})
    trades = entries.merge(exits, on=['strike', 'expiry_date', 'contract_d', 'date'])
    rename = {
        'date_x': 'entry_date',
        'close_x': 'entry_price',
        'spot_x': 'entry_spot',
        'date_y': 'exit_date',
        'close_y': 'exit_price',
        'spot_y': 'exit_spot'
    }
    trades = trades.rename(columns=rename)
    trades['pnl'] = trades.eval('exit_price-entry_price')
    trades['de'] = (trades['exit_date'] - trades['entry_date']).dt.days
    return trades


In [161]:
collect = []
for i,(opt,strike) in enumerate(options):
    data = ce if opt =='c' else pe
    temp = get_result(data, index, itm=strike,step=step)
    temp['name'] = i
    collect.append(temp)
res = pd.concat(collect)
del collect
res.pnl.describe()

count     762.000000
mean       -3.735761
std       155.439361
min      -496.150000
25%       -74.750000
50%       -40.275000
75%        25.925000
max      1564.200000
Name: pnl, dtype: float64

In [162]:
res2 = res.pivot(index='date', columns=['name'], values='pnl')
res3 = res2.iloc[:, 1] - res2.iloc[:, 0] + res2.iloc[:, 2]
res3.describe()

count    254.000000
mean      13.636024
std      164.413010
min     -358.050000
25%     -107.450000
50%       -4.000000
75%       94.962500
max      908.200000
dtype: float64